In [1]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import word2vec
import preprocessing

In [2]:
#Importazione del file relativo al post di stack già processati 

data = pd.read_csv('DB/Preprocessed_data.csv')

data

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.006907,class views django view points function proble...,Class views in Django view points to a functi...,class views django
1,2933,Create a directly-executable cross-platform GU...,user-interface|deployment|tkinter|release-mana...,0.040013,create directly executable cross platform gui ...,Create a directly-executable cross-platform GU...,create directly executable cross platform gui ...
2,19339,Transpose/Unzip Function (inverse of zip)?,list|matrix|transpose,0.093774,transpose unzip function list item tuples like...,Transpose/Unzip Function ? I have a list of 2...,transpose unzip function inverse zip
3,21961,Date/time conversion using time.mktime seems w...,datetime,0.000681,date time conversion using time mktime seems w...,Date/time conversion using time.mktime seems w...,date time conversion using time mktime seems w...
4,36806,Setup django with WSGI and apache,django|apache|mod-wsgi,0.001571,setup django wsgi apache sold mod wsgi apache ...,Setup django with WSGI and apache I have been ...,setup django wsgi apache
...,...,...,...,...,...,...,...
117290,69059275,split pandas frame into multiple frames based ...,pandas,-0.000208,split pandas frame multiple frames based timed...,split pandas frame into multiple frames based ...,split pandas frame multiple frames based timed...
117291,69059605,How to check if a column of DataFrame contain ...,pandas|dataframe|types,-0.000307,check column dataframe contain float type exam...,How to check if a column of DataFrame contain ...,check column dataframe contain float type
117292,69059907,How to increase/change a value in a dataset wi...,pandas|for-loop,-0.000505,increase change value dataset condition python...,How to increase/change a value in a dataset wi...,increase change value dataset condition python
117293,69059948,Check if elements in numpy array are between c...,numpy,-0.000307,check elements numpy array columns column nump...,Check if elements in numpy array are between c...,check elements numpy array columns column nump...


In [3]:
'''
Per poter fare il training word embeddings
In questa sezione importiamo l'intero corpo del DB dalla colonna unica che contiene: titolo, body e risposta
'''

documents = [_text.split() for _text in np.array(data.post_corpus)]

In [4]:
#data[1]

In [5]:
#Pre Train Word Embeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec(vector_size = 300, window = 7, min_count = 10, workers = 8 )

word_2_vec_model.build_vocab(documents)
words = list(word_2_vec_model.wv.index_to_key)
sizeVocab = len(words)

words
print('Size vocab:', sizeVocab)

#Train Word Embeddings
word_2_vec_model.train(documents, total_examples = len(documents), epochs = 32 )

#Salvataggio

word_2_vec_model.save('DB/word2vec_trained.bin')

Size vocab: 15109


In [2]:
word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')



print(word_2_vec_model.wv.most_similar("indexing"))
#print('-'*100)

[('slicing', 0.6815618872642517), ('slice', 0.5577464699745178), ('broadcasting', 0.518766462802887), ('indices', 0.4926108717918396), ('index', 0.47638317942619324), ('slices', 0.45064374804496765), ('iloc', 0.42658549547195435), ('indexer', 0.41888120770454407), ('indexes', 0.4133671522140503), ('indexed', 0.4125674068927765)]


In [7]:
word_2_vec_model.wv['slicing']

array([ 1.3841920e+00, -1.7659053e-01,  8.7667233e-01, -4.3570433e-02,
        6.9225299e-01,  1.9341147e+00,  4.6657300e-01,  1.3425446e+00,
       -1.4023435e+00, -1.2431734e-01,  1.5830641e+00, -9.4186622e-01,
        6.1650771e-01,  2.0873685e+00,  9.8512131e-01,  5.1262850e-01,
        9.0517038e-01, -1.9962704e+00,  1.8017937e+00, -7.9230583e-01,
        2.3961034e+00, -5.4882860e-01, -1.2418160e+00, -1.8935194e+00,
       -3.5349759e-01, -2.1409791e+00, -3.4493226e-01,  6.3769293e-01,
       -1.0234134e+00, -3.3752301e-01,  2.3441885e-01,  1.5775969e-02,
       -2.9225972e-01, -9.0227622e-01, -8.2381332e-01,  5.8105403e-01,
        4.0687388e-01, -1.0693189e+00,  1.6802024e+00, -1.6681864e+00,
       -5.1858403e-02, -2.0937836e+00, -1.0577613e+00,  4.1257098e-01,
        1.7852795e+00,  1.2886595e+00,  7.5054362e-02,  2.0849921e-01,
        1.3056468e+00,  2.6526424e-01,  7.7040285e-01, -4.3685371e-01,
        9.7175324e-01,  1.8037236e+00, -3.9648544e-02, -6.8700485e-02,
      